In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import re
import json
import numpy as np
from os import listdir
from os.path import isfile, join

In [ ]:
def season_loop(league,season):
    '''
    This fucntion scrapes an entire season of matches from Understat.com
    League: Name of the league. Character string.
    Season: Numeric of the year the season starts
    '''
    season_str = str(season) +'/' +str(season+1)[-2:]
    url_str = 'https://understat.com/league/' +league + '/' +str(season)
    response = requests.get(url_str)
    #'https://understat.com/league/')+league + '/' +str(season)
    datesData = re.search("datesData\s+=\s+JSON.parse\('([^']+)", response.text)
    decoded_string = bytes(datesData.groups()[0], 'utf-8').decode('unicode_escape')
    datesObj = json.loads(decoded_string)
    for element in datesObj:
        matchid = element['id']
        matchdate = element['datetime']
        hometeam = element['h'].get('title')
        awayteam = element['a'].get('title')
        fixture = hometeam + ' vs ' + awayteam
        homegoals = element['goals'].get('h')
        awaygoals = element['goals'].get('a')
        hxG = element['xG'].get('h')
        axG = element['xG'].get('a')
        df_match_json = pd.DataFrame({"season":[season_str],
                                      "matchid":[matchid],
                                      "matchdate":[matchdate],
                                      "fixture":[fixture],
                                      "hometeam":[hometeam],
                                      "awayteam":[awayteam],
                                      "homegoals":[homegoals],
                                      "awaygoals":[awaygoals],
                                      "hxG":[hxG],
                                      "axG":[axG]}
                                    )
        if element == datesObj[0]:
            df_json_season = df_match_json
        else:
            df_json_season =df_json_season.append(df_match_json,ignore_index = True)
    return df_json_season